In [1]:
import numpy as np
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
data = open('./data/input.txt','r')
data = list(data)
data_use = []
data_f = []
index = 0
temp = ''
for d in data:
    if d == "<start>\r\n":
        index = 1
        continue
    if d == '<end>\r\n':
        index = 0
    if index == 1:
        temp = temp + str(d)
    if index == 0:
        data_use.append(temp)
        temp = ''
#print(d)
for d in data_use:
    if len(d) > 40:
        data_f.append(d)
data_use = data_f
all_character = set()
counter = 0
for d in data_use:
    for dd in d:
        all_character.add(dd)
all_character = list(all_character)
nletter = len(all_character)

In [3]:
print(data_use[2])

XIV?me si?cle)
Z:Transcrit et/ou corrig? par Michel BELLON - 2005-03-27
Z:Pour toute observation mailto:galouvielle@free.fr
M:2/2
L:1/4
Q:1/4=110
K:Fmaj
V:1
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G2 zG|
w:Stel-__la splen-dens in mon-____te ut so-lis ra-_di-um Mi-__ra-cu-lis ser-ra-____to ex-au-di po-_pu-__lum  Prin-
Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 zG|Gd eg|ef/2d/2 ce|fd c/2B/2B/2A/2|G2 z2||
w:ci-pes et ma-gna-__ tes ex-stir-pe re-_gi-_a sae-cu-li po-tes-ta-__tes ob-ten-ta ve-_ni-_a
dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G|F4|dd/2c/2 dB| GB c2|_e/2d/2B/2c/2 AG|Bc c/2B/2 G/2A/4F/4|G4|]
w:Pe-__cca mi num pro-cla-____mant tu den tes pec-_to-ra Po-__pli te fle-xo cla-____mant hic A-ve Ma-_ri-__a
V:2
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d2 zd-|
d G2 G|cB A2|FG Bc|d2 zd-|d G2 G|cB A2|FG Bc|d2 z2|
G2 GB|dG F2|B2 cd|GF cd|f4| G2 GB|dG F2|B2 cd|GF Bc|d4|]



In [4]:
def lineToTensor(line):
    tensor = torch.zeros(len(line),nletter)
    position = np.zeros(len(line))
    for i, letter in enumerate(line):
        position[i] = all_character.index(letter)
        tensor[i][all_character.index(letter)] = 1
    position = torch.from_numpy(position[1:]).long()
    return autograd.Variable(tensor[:-1,:]).cuda(),autograd.Variable(position).cuda()

In [5]:
def batch_g(data,batch_size):
    lyric = np.random.randint(len(data))
    #print(len(data[lyric]))
    if len(data[lyric])-batch_size >0:
        interval_s = np.random.randint(len(data[lyric])-batch_size)
        return data[lyric][interval_s:interval_s+batch_size]
    else:
        return []

In [6]:
class LSTM(nn.Module):
    def __init__(self,letter_dim,hidden_dim):
        super(LSTM, self).__init__()
        self.letter_dim = letter_dim
        self.hidden_dim = hidden_dim
        self.hidden = self.init_hidden()
        
        self.lstm = nn.LSTM(letter_dim,hidden_dim)
        self.hidden2out = nn.Linear(hidden_dim,letter_dim)
    def init_hidden(self):
        return (autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, 1, self.hidden_dim)).cuda())
    def forward(self,line):
        lstm_out, self.hidden = self.lstm(line.view(len(line),1,-1), self.hidden)
        out_space = self.hidden2out(lstm_out.view(len(lstm_out),-1))
#         out_scores = F.softmax(out_space / 1.0,dim = 1)
        return out_space

In [7]:
hidden_dim = 100
batch_size = 30
lstm = LSTM(nletter,hidden_dim).cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm.parameters(), lr=0.001)

In [8]:
for epoch in range(1000):
    lstm.zero_grad()
    batch = batch_g(data_use,batch_size)
#     print(batch)
#     print('\n')
    if len(batch)>0:
        batch_in,batch_t = lineToTensor(batch)
        lstm.hidden = lstm.init_hidden()
        out_scores = lstm(batch_in)
        loss = loss_function(out_scores,batch_t)
        if epoch % 100 == 0:
            print(loss)
        loss.backward()
        optimizer.step()

Variable containing:
 4.5172
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.8141
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 4.0619
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.8350
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.2933
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.3618
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.9703
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.6403
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 3.1797
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 2.7703
[torch.cuda.FloatTensor of size 1 (GPU 0)]



In [9]:
print(F.softmax(out_scores / 1.0,dim = 1).data.cpu().numpy()[1])

[  9.40667540e-02   5.86557249e-03   5.36194025e-03   3.09963734e-03
   5.04027447e-03   3.77149368e-03   4.47191123e-04   1.14364119e-03
   1.31978467e-02   2.98417709e-03   3.49672744e-03   3.72165884e-03
   5.39380312e-03   1.19035591e-04   9.26656357e-04   4.11870889e-02
   1.10960258e-02   1.41291888e-02   4.20494610e-03   8.94330256e-03
   9.24673222e-04   2.30695866e-02   2.10815296e-03   2.33879522e-03
   4.10129927e-04   2.31001433e-02   1.47172501e-02   2.07788846e-03
   1.92853576e-03   6.31930027e-03   1.80170741e-02   2.87038623e-03
   3.10517289e-03   3.37854610e-03   9.93377180e-04   1.36338198e-03
   8.89697811e-04   2.08417978e-02   1.99023206e-02   2.68537668e-03
   1.59411691e-02   9.32564307e-03   1.71964103e-03   1.53673216e-04
   3.79079184e-03   4.51987889e-03   6.68260443e-04   1.51619271e-04
   2.95037660e-03   3.87398228e-02   3.24107497e-03   1.72441512e-01
   5.61616896e-03   2.70597953e-02   1.51797822e-02   1.00894552e-03
   1.98757020e-03   3.56636965e-03

In [10]:
present = torch.zeros(1,nletter)
present[0][all_character.index('X')] = 1
present = autograd.Variable(present).cuda()
lyric = []
lyric.append('X')
for num_generation in range(400):
    #print(lstm.forward(present).cpu())
    next_char_index = int(torch.distributions.Categorical(F.softmax(lstm(present)/1.0).data.cpu()).sample().numpy())
    next_char = all_character[next_char_index]
    lyric.append(next_char)
    present = torch.zeros(1,nletter)
    present[0][all_character.index(next_char)] = 1
    present = autograd.Variable(present).cuda()

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


In [22]:
print(str(lyric))

['X', '2', 'G', '2', ' ', 'B', '2', 'G', 'f', '|', 'e', '3', '@', 'd', 'c', 'e', 'd', ' ', '|', ' ', '(', '3', '.', 'g', 'z', 'f', 'g', '.', 'g', '3', ' ', 'd', 'c', 'a', '|', '\r', '\n', 'B', 'd', 'B', ' ', 'B', 'A', 'B', ' ', '|', ' ', 'G', '3', ' ', 'd', 'e', 'f', '|', '\r', '\n', 'G', 'd', 'c', 'A', '|', '\r', '\n', 'E', '2', ' ', 'A', '2', ')', '\r', '\n', 'N', '2', ' ', '|', ' ', '.', '7', '0', '1', ':', 'F', '3', ' ', 'C', ' ', 'D', ',', ' ', 'S', 'a', ' ', 'd', 'e', 'l', ' ', 'e', ',', ' ', 'H', 'b', 'i', 'n', '\r', '\n', 'R', ':', 's', 'o', 'l', ' ', 'e', 'e', '\r', '\n', 'R', ':', 's', 'e', 'e', 'd', 'a', 'n', 'o', ' ', 'b', 'a', 'l', 'e', ' ', 'd', 'o', 'q', 't', 'e', 'r', 'i', 'e', 't', ' ', 'C', 'o', 'r', 'V', 'a', 'g', '?', ':', '\r', '\n', 'S', ':', 'C', 'a', 'n', 'd', 'e', 'c', 'd', ':', ' ', '\\', 'i', 'd', 't', 'r', 'e', '@', 'b', 's', 'e', 'r', 'v', 'e', '\r', '\n', 'R', ':', 'C', 'h', 'a', 'n', ' ', 's', 'h', 'i', 'm', ' ', 't', 'o', 'u', 't', ' ', 'o', 'l', 'i', 'o

In [23]:
Music = ""
for char in lyric:
    Music = Music + char
print(Music)

X2G2 B2Gf|e3@dced | (3.gzfg.g3 dca|
BdB BAB | G3 def|
GdcA|
E2 A2)
N2 | .701:F3 C D, Sa del e, Hbin
R:sol ee
R:seedano bale doqteriet CorVag?:
S:Candecd: \idtre@bserve
R:Chan shim tout olioek Wira(
T:aadlwe
P:Slid toz Woska:
T:+irnellin s0macesc, Pour Michens Bf anye Roucs
R:slide
Z:id:hn-sereg- a-77
M:C:
K:G
ABBA2 :EFE|GBBcd Adcrameldub, The
Z:id:hn-reel-178
M:C|
K:C
c2dB BcA2|
